In [459]:

# Parameters
import numpy as np
P = 50 # Number of states (and sensors for simplicity)
N = 50 # Number of sensors
R = np.eye(N)
#x_hat = np.zeros(P).T
A = np.zeros((P, P))  # Initialize a P x P matrix of zeros

for i in range(P):
    for j in range(P):
        if i == j:
            A[i, j] = 3/4  # Diagonal elements
        elif (i - 2 * j) % 4 == 0:
            A[i, j] = -1/8  # Specific off-diagonal interactions
        else:
            A[i, j] = 0  # No interaction
Q = np.zeros((P, P))  # Initialize a P x P matrix of zeros

for i in range(P):
    for j in range(P):
        if i == j:
            Q[i, j] = (11 + (i % 5)) / 5  # Diagonal elements based on i
        elif (i - j) % 6 == 0:
            Q[i, j] = 1  # Specific off-diagonal interactions
        else:
            Q[i, j] = 0  # No interaction for other cases

#x = np.zeros(P).T
x_hat = np.zeros((P,1))
P_k = np.zeros((P,P))
x = np.zeros((P,1))
x0 = np.zeros((P,1))
H = np.eye(N, P)  # Observation matrix
#P_K = np.zeros((P,P))  # Initial estimation covariance matrix
epsilon = np.array([0.02 * ((n + 1) / N) for n in range(N)])
v = np.random.multivariate_normal(np.zeros(N), Q).reshape(N,1)
w = np.random.multivariate_normal(np.zeros(N), R).reshape(N, 1)

import numpy as np
def update_covariance_successful_update(n,P_k,x,x_hat):
    #x = A @ x+ v
    x_hat_pred = A @ x_hat # predicted state estimate
    
    P_pred = A * P_k * A.T + Q
    y = H @ x + w
    
    S = H * P_k * H.T + R
    one_hot_vector = np.zeros(N)
   
    one_hot_vector[n] = 1
    one_hot_vector = one_hot_vector.reshape(1, N)
    one_hot__trans = one_hot_vector.reshape(N, 1) 
    #print(one_hot__trans.shape)
    g_K_n = P_pred * H.T @ one_hot__trans /S[n,n]
    x_hat = x_hat_pred  + g_K_n*(y[n]-(H @ x_hat_pred)[n])
    print(g_k_n)
    #x_hat = x_hat_pred  + g_K_n*(y[n]-x_hat_pred[n])
    mse = (x-x_hat).T @ (x-x_hat)
    #mse = (x[n]-x_hat[n])**2
    
    P_k = (np.eye(P) - (g_K_n @ one_hot_vector @ H) ) * P_pred

    return P_k, x_hat,mse





In [462]:

for i in range(50):
    x = A @ x0 + v
    y = H @ x + w
    
    #print(f'p_k before update is \n {P_k}')
    #print(f'x before update is \n {x}')
    #print(f'y before update is \n {y}')
    #print(f'x_hat before update is \n {x_hat}')
    P_k,x_hat,mse= update_covariance_successful_update(i,P_k,x,x_hat)
    #print(f'p_k after update is \n {P_k}')
    #print(f'x after update is \n {x}')
    #print(f'y before update is \n {y}')
    #print(f'x_hat after update is \n {x_hat}')
    print(f'mse is {mse}')
    x0=x
    

mse is [[15940.52681643]]
mse is [[17578.84697282]]
mse is [[17156.90881291]]
mse is [[16287.47182358]]
mse is [[16599.89741406]]
mse is [[18357.48799711]]
mse is [[16549.50421013]]
mse is [[15783.54192996]]
mse is [[15819.0804312]]
mse is [[17887.40491861]]
mse is [[16007.57721832]]
mse is [[15667.20009994]]
mse is [[16104.84864473]]
mse is [[14343.58170138]]
mse is [[14321.16889284]]
mse is [[12357.55528018]]
mse is [[13998.73947436]]
mse is [[15674.4823685]]
mse is [[13782.47256088]]
mse is [[11799.04588411]]
mse is [[12972.14344125]]
mse is [[13993.44368048]]
mse is [[13567.0088653]]
mse is [[12889.26841776]]
mse is [[13973.61226117]]
mse is [[14384.46135098]]
mse is [[14098.32984834]]
mse is [[13191.52553576]]
mse is [[14539.76933391]]
mse is [[15413.54832165]]
mse is [[14555.15570127]]
mse is [[13104.87829485]]
mse is [[13459.45202346]]
mse is [[14295.66861525]]
mse is [[13228.91485982]]
mse is [[12383.36638405]]
mse is [[13315.98803037]]
mse is [[13412.66722376]]
mse is [[13838.

In [450]:

import numpy as np
from scipy.stats import norm
from scipy.integrate import quad

def compute_Pn_E(Rnn,Pnn):
    theta = np.sqrt(np.maximum(Pnn, 0.001))

    """Compute the error probability based on R and theta."""
    
    return 1 - 2 * norm.cdf((-theta) / np.sqrt(Rnn))

def calculate_Pn_Ex(Pn_E, epsilon_n):

    return Pn_E / (Pn_E + (1 - Pn_E) * epsilon_n)

def calculate_Pn_X(Pn_E,epsilon_n):
   
   return epsilon_n + (1-epsilon_n)*Pn_E

def posterior_pdf(zn, Pnn, Rnn):
    """Compute the posterior PDF p(zn|ξ) for given parameters."""
    theta = np.sqrt(np.maximum(Pnn, 1))
    Pn_E= compute_Pn_E(Rnn,Pnn)
    pdf_zn = norm.pdf(zn/np.sqrt(np.maximum(Pnn, 0.0001)))
    cdf_plus = norm.cdf((theta - zn) / np.sqrt(Rnn))
    cdf_minus = norm.cdf((-theta - zn) / np.sqrt(Rnn))
    
    return pdf_zn * (cdf_plus - cdf_minus) /Pn_E

#def integrand(zn, Pnn, Rnn):
#        return posterior_pdf(zn,Pnn,Rnn) * zn**2


def integral_zn_squared(Pnn, Rnn):
    """Compute the integral for updating P_{m,n} based on p(zn|ξ)."""
    integrand = lambda zn: posterior_pdf(zn,Pnn,Rnn) * zn**2
    integral_result, _ =  quad(integrand, -50, 50)
    
    return integral_result

def update_covariance_matrix(P_k, R, epsilon):
    """Update the covariance matrix P based on the specified parameters."""
    
    P_updated = np.zeros_like(P_k)
    P_dim = P_k.shape[0]
    Pn_Ex  = np.zeros(P_k.shape[0])
    
    for n in range(P_dim):
        Pnn = P_k[n, n]
        Rnn = R[n, n]
        integral_result = integral_zn_squared( Pnn,Rnn)
        Pn_E= compute_Pn_E(Rnn, Pnn)
        Pn_Ex[n] = calculate_Pn_X(Pn_E,epsilon[n])
        for m in range(P_dim):
            Pmn = P_k[m, n]
            P_updated[m, n] = (Pmn/np.maximum(Pnn, 0.001))*integral_result

        
    #print(f'pn_ex is {(1 - Pn_Ex)}')        
    #print(f'pn_ex * Pis {(1 - Pn_Ex) * P}')    
    return  (Pn_Ex * P_updated + (1 - Pn_Ex) * P_k)



def update_covariance_failed_update(P_k,R, epsilon):
    
    P_k = update_covariance_matrix(P_k,R,epsilon)
  
    return P_k


i=1
for n in range(i):
  print(f'before update \n : {P_k}')
  multiplier = 3
  P_k = update_covariance_failed_update(P_k, R, epsilon)
  print(f'after update \n : {P_k}')
  #print(f'pn_X for node {n} =  : {Pn_X[n]}')



before update 
 : [[ 6.48043287 -0.45611953  3.26051333 ... -0.61277495  3.25571754
  -0.52717332]
 [-0.56435351  2.63613014  0.36359058 ... -0.19941102 -0.60818544
   0.90430545]
 [ 3.61178381  0.32551861 -1.68301871 ...  0.38811645  3.91333316
   0.37607826]
 ...
 [-0.73793159 -0.19408487  0.42193095 ...  3.06295002 -0.79400399
  -0.22408286]
 [ 2.98170337 -0.45017479  3.23540589 ... -0.60384477  8.79911378
  -0.52032691]
 [-0.61453838  0.85199721  0.39576561 ... -0.21691478 -0.66229909
   3.57090115]]
after update 
 : [[ 3.87867946 -0.2421824   3.17444605 ... -0.32367516  2.16652442
  -0.2801797 ]
 [-0.3377778   1.39968646  0.35399293 ... -0.10533132 -0.40471835
   0.48061618]
 [ 2.16173086  0.1728382  -1.63859232 ...  0.20500781  2.60413619
   0.19987638]
 ...
 [-0.44166804 -0.1030518   0.4107933  ...  1.61788732 -0.52837171
  -0.11909455]
 [ 1.78461406 -0.23902597  3.15000138 ... -0.31895812  5.85538969
  -0.276541  ]
 [-0.36781453  0.45237864  0.38531865 ... -0.11457701 -0.440728

In [451]:
import numpy as np
""" 
def polling_decision(P_k,x_hat, R, epsilon, N, M):
    # Create copies of P_Ks for simulation purposes
    P_k_chi = np.copy(P_k)  # Covariance if update fails
    P_k_n = np.copy(P_k)    # Covariance if update succeeds
    #print(P_k_n) 
   
    # Simulate successful update using existing function on copies
    x_hat_temp = np.copy(x_hat)
    #x_copy = np.copy(x)  # Temporary state estimates for simulation
    expected_reduction = np.zeros(N)
    P_k_chi = update_covariance_failed_update(P_k_chi , R, epsilon)
    Pn_X = np.zeros(P_k.shape[0])
    for n in range(N):
       
        P_k_n,x_hat = update_covariance_successful_update(n, P_k_n, x_hat_temp,y)
        #print(P_k_n.shape)
        Rnn= R[n,n]
        Pnn= P_k_n[n,n]
        
        Pn_E = compute_Pn_E(Rnn, Pnn)
        Pn_X[n] = calculate_Pn_X(Pn_E,epsilon[n])
        #expected_reduction[n] = np.trace(P_k) - Pn_X[n] * np.trace(P_k_chi) - (1 - Pn_X[n]) * np.trace(P_k_n)

        ###   if last_polled[n]:
            ###expected_reduction[n] = -np.inf
    
    #top_sensors = np.argsort(-expected_reduction)[:M]
        #print(n)
        #print(P_k_n.shape)
    expected_reduction = np.trace(P_k) - Pn_X * np.trace(P_k_chi) - (1 - Pn_X) * np.trace(P_k_n)      
    # Simulate failure scenarios on copies
    #last_polled[:] = False  # Reset all to false
    #last_polled[top_sensors] = True  # Set true for just polled sensors
 
    # Calculate the expected reduction for each sensor


    # Select the top M sensors based on the expected MSE reduction
    #print(expected_reduction)
    top_sensors = np.argsort(-expected_reduction)[:M]
    return top_sensors.tolist()
"""
import numpy as np

def polling_decision(P_k, x_hat, R, epsilon, N, M, last_polled, decay_factor):
    # Create copies of P_Ks for simulation purposes
    P_k_chi = np.copy(P_k)  # Covariance if update fails
    P_k_n = np.copy(P_k)    # Covariance if update succeeds
    
    # Temporary state estimates for simulation
    x_hat_temp = np.copy(x_hat)
    
    # Array to store expected reduction in MSE for each sensor
    expected_reduction = np.zeros(N)
    
    # Update covariances assuming no packet received
    P_k_chi = update_covariance_failed_update(P_k_chi, R, epsilon)
    
    # Compute expected reduction and simulate successful update
    for n in range(N):
        Rnn = R[n, n]
        Pnn = P_k_n[n, n]
        Pn_E = compute_Pn_E(Rnn, Pnn)
        Pn_X = calculate_Pn_X(Pn_E, epsilon[n])
        if not last_polled[n]:  # Only consider sensors not recently polled
            P_k_n, x_hat = update_covariance_successful_update(n, P_k_n, x_hat_temp,y)

            expected_reduction[n] = np.trace(P_k) - Pn_X * np.trace(P_k_chi) - (1 - Pn_X) * np.trace(P_k_n)
        else:
            # Apply decay to recently polled sensors
            expected_reduction[n] = (np.trace(P_k) - Pn_X * np.trace(P_k_chi) - (1 - Pn_X) * np.trace(P_k_n)) * decay_factor

    # Select the top M sensors based on the adjusted expected MSE reduction
    top_sensors = np.argsort(-expected_reduction)[:M]

    # Update last polled status
    new_last_polled = np.zeros(N, dtype=bool)
    new_last_polled[top_sensors] = True

    return top_sensors.tolist(), new_last_polled





In [335]:

# Initialize parameters
N = 50  # Number of sensors
M = 5   # Number of sensors to poll each time
P_k = np.eye(N)  # Example covariance matrix
x_hat = np.zeros(N)  # Example state estimates
R = np.eye(N)  # Example measurement noise covariance matrix
epsilon = np.random.rand(N)  # Example failure probabilities
last_polled = np.zeros(N, dtype=bool)  # Track sensors polled in the last decision
decay_factor = 0  # Decay factor for recently polled sensors

# Run polling decision multiple times to see the effect
for _ in range(3):
    selected_sensors, last_polled = polling_decision(P_k, x_hat, R, epsilon, N, M, last_polled, decay_factor)
    print("Selected sensors:", selected_sensors)


Selected sensors: [46, 26, 42, 18, 38]
Selected sensors: [20, 44, 40, 14, 30]
Selected sensors: [34, 16, 26, 10, 22]


In [336]:
import numpy as np
num_steps = 20
P = 50  # Number of states (and sensors for simplicity)
N = 50  # Number of sensors
R = np.eye(N)
#x_hat = np.zeros(P).T
A = np.zeros((P, P))
Q = np.zeros((P, P))
for i in range(P):
    for j in range(P):
        if i == j:
            A[i, j] = 3/4  # Diagonal elements
        elif (i - 2 * j) % 4 == 0:
            A[i, j] = -1/8  # Specific off-diagonal interactions
        else:
            A[i, j] = 0  # No interaction
Q = np.zeros((P, P))  # Initialize a P x P matrix of zeros

for i in range(P):
    for j in range(P):
        if i == j:
            Q[i, j] = (11 + (i % 5)) / 5  # Diagonal elements based on i
        elif (i - j) % 6 == 0:
            Q[i, j] = 1  # Specific off-diagonal interactions
        else:
            Q[i, j] = 0  # No interaction for other cases
#x = np.zeros(P).T
x_hat = np.zeros((P,1))
M=10
P_k = np.eye(P)
x = np.zeros((P,1))
H = np.eye(N, P)  # Observation matrix
#P_K = np.zeros((P,P))  # Initial estimation covariance matrix
epsilon = np.array([0.02 * ((n + 1) / N) for n in range(N)])
v = np.random.multivariate_normal(np.zeros(N), Q).reshape(N,1)
w = np.random.multivariate_normal(np.zeros(N), R).reshape(N, 1)
multiplier = 1

last_polled = np.zeros(N, dtype=bool)  # Track sensors polled in the last decision
decay_factor = 0  # Decay factor for recently polled sensors


def main_simulation(x0, x_hat, num_steps, P_k, N, R, epsilon, M,last_polled):
    # Initialize state estimates, covariance matrices, and tracking counters
    x = x0
    #y = H @ x + w
    polled_count = np.zeros(N)  # Track number of times each sensor was polled
    transmitted_count = np.zeros(N)  # Track number of times each sensor transmitted

    mse_records = []  # To record MSE after each simulation step

    for step in range(num_steps):
        # Step 1: Determine which sensors to poll based on the MSE reduction strategy
        x = A@ x + v
        selected_sensors,last_polled = polling_decision(P_k, x_hat, R, epsilon, N, M, last_polled, decay_factor)
         # Increment polling count for selected sensors
        
        y = H @ x + w

        # Step 2: Process measurements for each selected sensor
        for sensor_index in selected_sensors:
            polled_count[selected_sensors] += 1 
            print(f'pulled sensor index is {sensor_index}')
            
            # Calculate dynamic threshold
            theta = multiplier * np.sqrt(np.maximum(P_k[sensor_index, sensor_index],1))
            a = x_hat[sensor_index].item() - theta
            b = x_hat[sensor_index].item() + theta
            y_value = y[sensor_index].item()

            
            if a <= y_value <= b: #and np.random.rand() > epsilon[sensor_index]:
               P_k = update_covariance_failed_update(P_k,R, epsilon)

               print(f'no update received')
            else :
               P_k,x_hat =update_covariance_successful_update(sensor_index, P_k,x_hat,y)
               transmitted_count[sensor_index] += 1  # Increment transmission count
               print(f'update received')
        mse = (x - x_hat).T @ (x - x_hat)
               #mse =(x[sensor_index].item() - x_hat[sensor_index].item())**2
               
        mse_records.append(mse)
        print(f'the mse {mse}')

        print(f'num_steps is {step}')


 
        

    # Average MSE over all simulation steps
    average_mse = np.mean(mse_records)

    E_max = 162000  
    E_t = 0.05     
    E_s = 0.01    
    E_w = 0.01    
    E_0 = 0.001   
    fw = polled_count / (num_steps)
    ft = transmitted_count / (num_steps)
    average_lifetime_hours = np.mean([
        E_max / (ft[n] * E_t + fw[n] * (E_s + 3 * E_w) + (1 - fw[n]) * E_0)
        for n in range(N)
    ]) / 3600  # Convert seconds to hours
    average_lifetime_years = average_lifetime_hours / 8760  # Hours per year

    return average_mse, mse_records, average_lifetime_years

# Example usage of the simulation with provided parameters

average_mse, mse_records, average_lifetime_years = main_simulation(x,x_hat, num_steps, P_k, N, R, epsilon, M,last_polled)
print("Average MSE over simulation:", average_mse)
print("Estimated average sensor lifetime (years):", average_lifetime_years)


pulled sensor index is 46
update received
pulled sensor index is 30
update received
pulled sensor index is 22
update received
pulled sensor index is 34
no update received
pulled sensor index is 18
update received
pulled sensor index is 42
no update received
pulled sensor index is 38
no update received
pulled sensor index is 14
no update received
pulled sensor index is 26
update received
pulled sensor index is 10
no update received
the mse [[241.60660379]]
num_steps is 0
pulled sensor index is 44
no update received
pulled sensor index is 24
no update received
pulled sensor index is 32
no update received
pulled sensor index is 48
no update received
pulled sensor index is 40
no update received
pulled sensor index is 36
no update received
pulled sensor index is 28
update received
pulled sensor index is 20
no update received
pulled sensor index is 16
no update received
pulled sensor index is 12
no update received
the mse [[683.95568256]]
num_steps is 1
pulled sensor index is 24
no update re

KeyboardInterrupt: 